In [85]:
import pandas as pd
import matplotlib.pyplot as plt

In [86]:
# Importing the dataset
df = pd.read_csv("megaGymDataset.csv")
df

,Unnamed: 0,Title,Desc,Type,BodyPart,Equipment,Level,Rating,RatingDesc
0,0,Partner plank band row,The partner plank band row is an abdominal exe...,Strength,Abdominals,Bands,Intermediate,0.0,NaN
1,1,Banded crunch isometric hold,The banded crunch isometric hold is an exercis...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
2,2,FYR Banded Plank Jack,The banded plank jack is a variation on the pl...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
3,3,Banded crunch,The banded crunch is an exercise targeting the...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
4,4,Crunch,The crunch is a popular core exercise targetin...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2913,2913,EZ-bar skullcrusher-,The EZ-bar skullcrusher is a popular exercise ...,Strength,Triceps,E-Z Curl Bar,Intermediate,8.1,Average
2914,2914,Lying Close-Grip Barbell Triceps Press To Chin,NaN,Strength,Triceps,E-Z Curl Bar,Beginner,8.1,Average
2915,2915,EZ-Bar Skullcrusher - Gethin Variation,The EZ-bar skullcrusher is a popular exercise ...,Strength,Triceps,E-Z Curl Bar,Intermediate,NaN,NaN
2916,2916,TBS Skullcrusher,The EZ-bar skullcrusher is a popular exercise ...,Strength,Triceps,E-Z Curl Bar,Intermediate,NaN,NaN


In [88]:
#Cheking if there is any NULL or missing values
df.isna().sum()

Unnamed: 0       0
Title            0
Desc          1550
Type             0
BodyPart         0
Equipment        0
Level            0
Rating        1887
RatingDesc    2056
dtype: int64

In [89]:
# Removing columns with lots of nonvalues
df = df.drop('Rating', axis=1)
df = df.drop('RatingDesc', axis=1)
# Removing all rows containing nonvalues in description
df = df[df['Desc'].notna()]
# Removing ID column
df.pop(df.columns[0])



0          0
1          1
2          2
3          3
4          4
        ... 
2913    2913
2914    2914
2915    2915
2916    2916
2917    2917
Name: Unnamed: 0, Length: 2918, dtype: int64

In [90]:
# Checking datatypes
df.dtypes

Title        object
Desc         object
Type         object
BodyPart     object
Equipment    object
Level        object
dtype: object

In [94]:
# Merging columns for cosign similarity and dropping excess columns
df["Merged"] = df["Type"].astype(str) + '|' + \
  df["BodyPart"].astype(str) + '|' + df["Equipment"].astype(str) + '|' + \
  df["Level"]

df = df.drop('Type', axis=1)
df = df.drop('BodyPart', axis=1)
df = df.drop('Equipment', axis=1)
df = df.drop('Level', axis=1)

In [106]:
# The merged columns
df["Merged"]

0           Strength|Abdominals|Bands|Intermediate
1           Strength|Abdominals|Bands|Intermediate
2           Strength|Abdominals|Bands|Intermediate
3           Strength|Abdominals|Bands|Intermediate
4           Strength|Abdominals|Bands|Intermediate
                           ...                    
2913    Strength|Triceps|E-Z Curl Bar|Intermediate
2914        Strength|Triceps|E-Z Curl Bar|Beginner
2915    Strength|Triceps|E-Z Curl Bar|Intermediate
2916    Strength|Triceps|E-Z Curl Bar|Intermediate
2917    Strength|Triceps|E-Z Curl Bar|Intermediate
Name: Merged, Length: 2918, dtype: object

In [97]:
# Converting values of the merged column into vectors

from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
count_matrix = count.fit_transform(df.loc[:,"Merged"])

liste = count_matrix.toarray()

In [98]:
# Cosine similarity
from sklearn.metrics.pairwise import cosine_similarity
sim_matrix = cosine_similarity(count_matrix, count_matrix)

In [99]:
#sim_matrix

In [107]:
def recommender(data_frame, exercise_id, sim_matrix):
    sim_df = pd.DataFrame(sim_matrix[exercise_id],
                         columns=["similarity"])
    exercise_titles = data_frame.loc[:, "Title"]
    exercise_rec = pd.concat([sim_df, exercise_titles], axis = 1)

    exercise_rec = exercise_rec.sort_values(by=["similarity"], ascending = False)

    return exercise_rec.iloc[1:20,:]

In [108]:
# Exercises similar to bench press
#recommender(df, 916, sim_matrix)

# Exercises similar to crunches
recommender(df, 200, sim_matrix)

,similarity,Title
416,1.0,30 Hanging Oblique Crunch
409,1.0,30 Decline Reverse Crunch
410,1.0,Decline sit-up twist
411,1.0,30 Flat Bench Leg Raise
412,1.0,30 Flutter Kick
413,1.0,Chin-up crunch
414,1.0,30 Gorilla Chin Oblique Crunch
415,1.0,Half Turkish get-up
417,1.0,30 Hollow-Body Hold
426,1.0,30 Hanging Toes-To-Bar
